In [1]:
import sys
import os
sys.path.append(os.path.abspath('../src'))
sys.path.append(os.path.abspath('../Dataset'))
sys.path.append(os.path.abspath('..'))

import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from func_preprocesamiento import preprocesar, preprocesar_binarios, procesar_antiguedad, preprocesar_categoricos

In [2]:
dev_df = pd.read_csv('../Dataset/raw/dev.csv')
bins = pd.qcut(dev_df['precio_pesos_constantes'], q=5, labels=False)  # 4 cuartiles
train_dataset, val_dataset = train_test_split(dev_df, train_size=0.2, random_state=42, stratify=bins)

X_train = train_dataset.drop(columns=['precio_pesos_constantes'])
y_train = train_dataset['precio_pesos_constantes']

/tmp/ipykernel_16322/1641456509.py:1: DtypeWarning: Columns (8,33) have mixed types. Specify dtype option on import or set low_memory=False.
  dev_df = pd.read_csv('../Dataset/raw/dev.csv')


In [3]:
df = preprocesar(train_dataset)
direccion = '../Dataset/Procesado/train_procesado.csv'
df.to_csv(direccion, index=False)

Columna a predecir: ITE_ADD_NEIGHBORHOOD_NAME_encoded
Columna a predecir: SalonDeUsosMul
Columna a predecir: AireAC
Columna a predecir: Estacionamiento
Columna a predecir: Seguridad
Columna a predecir: AreaParrillas
Columna a predecir: CanchaTennis
Columna a predecir: AreaCine
Columna a predecir: Gimnasio
Columna a predecir: Amoblado
Columna a predecir: STotalM2
Columna a predecir: SConstrM2
Columna a predecir: Antiguedad


## DATASET NUMERICO

In [4]:
#ARMAMOS EL DATASET NUMERICO SACANDO LAS COLUMNAS CATEGORICAS
columna = ['ITE_ADD_CITY_NAME', 'ITE_ADD_STATE_NAME', 'ITE_ADD_NEIGHBORHOOD_NAME', 'ITE_TIPO_PROD', 'TIPOPROPIEDAD'] #SACARLAS PORQUE SON CATEGORICAS
direccion = '../Dataset/Procesado/train_procesado_numerico.csv'
df = df.drop(columns=columna)
df.to_csv(direccion, index=False)
train_procesado = pd.read_csv(direccion)


In [5]:
# import pandas as pd

# # Cargar el DataFrame
# dev_procesado = pd.read_csv('../Dataset/Procesado/dev_procesado_numerico.csv')

# # Calcular la matriz de correlación
# correlacion = dev_procesado.corr()

# # Calcular la correlación de todas las columnas con la columna 'precio_pesos_constantes', excluyendo la misma columna
# correlacion_precio = correlacion['precio_pesos_constantes'].drop(labels=['precio_pesos_constantes']).sort_values(ascending=False)

# # Imprimir las columnas que más influyen en el precio
# print("Columnas que más influyen en el precio:")
# print(correlacion_precio)